In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from tensorflow.keras.preprocessing import sequence
import tensorflow as tfs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D,LeakyReLU
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2, random

In [ ]:
height=32
width=32
depth=3

inputShape = (height, width, depth)

In [ ]:
# Prepare the train and test dataset.
from tensorflow import keras
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 32, 32, 3))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 32, 32, 3))


169001437/169001437 [==============================] - 6s 0us/step


In [ ]:
from  keras.utils import np_utils
# y_train = np_utils.to_categorical(y_train)
# y_test= np_utils.to_categorical(y_test)
# print(y_train[0])
print(y_train.shape,y_test.shape)
print(x_train.shape)
# print(y_train[0],x_train[0])

ImportError: ignored

In [ ]:
from sklearn.utils import shuffle
x_train, y_train= shuffle(x_train, y_train, random_state=22)
X=[]
Y=[]
j=0
for i in range(40):
  X.append(x_train[j:j+2500])
  Y.append(y_train[j:j+2500])
  j+=1200



In [ ]:
X=np.array(X)
Y=np.array(Y)
print(X.shape,Y.shape)

In [ ]:
from tensorflow.keras.layers import BatchNormalization
class MODEL:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Conv2D(input_shape=inputShape, kernel_size=(2, 2), padding='same', strides=(2, 2), filters=32))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
        model.add(Conv2D(kernel_size=(2, 2), padding='same', strides=(2, 2), filters=64))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(100, activation='softmax'))
        return model

In [ ]:
from sklearn.metrics import accuracy_score
def test_model(X_test, Y_test,  model, comm_round):
    model.compile(  optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
    # loss,acc=model.evaluate(x_test,y_test)
    acc,loss=model.evaluate(x_test,y_test)
    return acc, loss
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

In [ ]:
from tensorflow import keras

In [ ]:
global1= MODEL()
global_model = global1.build()
comms_round = 10
# print(trainy[2].shape)
epoch=5
total_clusters=2
total_clients=40
clients_per_cluster=int(total_clients/total_clusters)
X=np.reshape(X,(total_clusters,clients_per_cluster,2500,32,32,3))
Y=np.reshape(Y,(total_clusters,clients_per_cluster,2500))
print(X.shape,Y.shape)

from tensorflow.keras import backend as K
for comm_round in range(comms_round):
  global_weights = global_model.get_weights()
  scaled_cluster_weight_list = list()
  index=list({0,1,2,3,4})
  #random.shuffle(index)
  # print(index)
  print("\nCommunication Round:" ,comm_round)
  for clstr in range(total_clusters):
    scaled_local_weight_list = list()
    cluster_model=MODEL().build()
    cluster_model.compile(optimizer=keras.optimizers.Adam(),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=[keras.metrics.SparseCategoricalAccuracy()],)

    cluster_model.set_weights(global_weights)
    cluster_weights = cluster_model.get_weights()
    for ind in range(clients_per_cluster):

      local = MODEL()
      local_model=local.build()
      local_model.compile(optimizer=keras.optimizers.Adam(),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=[keras.metrics.SparseCategoricalAccuracy()],)
      local_model.set_weights(cluster_weights)

      history=local_model.fit(X[clstr][ind],Y[clstr][ind], epochs=epoch,verbose=0)#,validation_data=(x_test,y_test))

      scaling_factor=1.0/clients_per_cluster #1/no.ofclients
      scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
      scaled_local_weight_list.append(scaled_weights)
      K.clear_session()
    average_cluster_weights = sum_scaled_weights(scaled_local_weight_list)
    cluster_model.set_weights(average_cluster_weights)

    scaling_cluster_factor=1.0/total_clusters #1/no.ofclients
    scaled_cluster_weights = scale_model_weights(cluster_model.get_weights(), scaling_cluster_factor)
    scaled_cluster_weight_list.append(scaled_cluster_weights)
    K.clear_session()
    cluster_loss, cluster_acc = test_model(x_test,y_test, cluster_model, comm_round)
    print("CLUSTER ",clstr,"ACC: ",cluster_acc,"CLUSTER ",clstr,"LOSS: ",cluster_loss)

  average_weights = sum_scaled_weights(scaled_cluster_weight_list)
  print(len(scaled_cluster_weight_list))
  #global_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  global_model.set_weights(average_weights)
  #val_acc,val_loss=global_model.evaluate(x_train,y_train)
  #print(val_loss,val_acc)
  global_loss, global_acc = test_model(x_test,y_test, global_model, comm_round)
  print("GLOBAL ACCURACY",global_acc,"GLOBAL LOSS",global_loss)








In [ ]:
print(global_loss)

In [ ]:
print(global_acc)